In [ ]:
import datasets
from src.make_datasets import make_sentence_files

# load the japanese dataset , sample and split into sentences
dataset_ja = datasets.load_dataset("cc100", lang="ja")
dataset_ja["train"][961563]
make_sentence_files(dataset_ja["train"])

In [1]:
import datasets
from src.make_datasets import make_sentence_files

# load the english dataset , sample and split into sentences
dataset_en = datasets.load_dataset("cc100", lang="en")
dataset_en["train"][961563]
make_sentence_files(dataset_en["train"],data_dir = cfg.EN_SENTENCES_DIR)

Using custom data configuration en-lang=en


0 examples [00:00, ? examples/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

writing to  /work/data/jp_sentences/sent_4.txt


 64%|███████████████████████████████████████████████████████████▏                                 | 6367585/10000000 [02:29<01:33, 38862.80it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [03:49<00:00, 43534.79it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [03:44<00:00, 44550.65it/s]


writing to  /work/data/jp_sentences/sent_33.txt


  7%|██████▍                                                                                       | 679611/10000000 [00:15<03:37, 42788.91it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [03:43<00:00, 44814.83it/s]


writing to  /work/data/jp_sentences/sent_38.txt


 88%|█████████████████████████████████████████████████████████████████████████████████▊           | 8800105/10000000 [03:16<00:25, 46443.21it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print("test")
dataset_en["train"][961563]

In [ ]:
import glob
import cfg
from random import sample
from tqdm import tqdm
import os

num_files = 7

# read text files as a list
jp_sentence_files = glob.glob(cfg.JP_SENTENCES_DIR + "/*.txt")

# sample num_files
sampled_files=sample(jp_sentence_files, num_files)
print(sampled_files)

#read all the lines as a list
all_lines = []
for filename in sampled_files:
    with open(filename) as f:
        lines = f.read().splitlines()
        
    all_lines.extend(lines)

print("number of lines sampled:", len(all_lines))

#combine into a single file

tempfile = os.path.join(cfg.DATA_DIR, "temp.txt")
with open(tempfile, "w") as f:

            for sentence in tqdm(all_lines):
                
                # remove newlines
                line = sentence.strip()

                # do not save empty items such as
                if sentence != []:

                    f.writelines(sentence + '\n')

In [ ]:
import sentencepiece as spm
import time

#spm.SentencePieceTrainer.train(input=jp_sentence_files, 
#                               model_prefix='m', vocab_size=1000)

vocab_size = 32000

start = time.time()
model_prefix = "cc100_jp" + "_vocab_" + str(vocab_size)
spm.SentencePieceTrainer.train(input=tempfile
                                       , model_prefix=model_prefix
                                       , vocab_size=vocab_size
                                       , character_coverage = 0.9995
                                       , num_threads=60
                                       , train_extremely_large_corpus=True
                                      ) 
print("Trained {} in {} seconds".format(model_prefix, time.time()-start))

In [ ]:
import glob
import cfg
from random import sample
from tqdm import tqdm
import os

num_files = 7

# read text files as a list
en_sentence_files = glob.glob(cfg.EN_SENTENCES_DIR + "/*.txt")

# sample num_files
sampled_files=sample(en_sentence_files, num_files)
print(sampled_files)

#read all the lines as a list
all_lines = []
for filename in sampled_files:
    with open(filename) as f:
        lines = f.read().splitlines()
        
    all_lines.extend(lines)

print("number of lines sampled:", len(all_lines))

#combine into a single file

tempfile = os.path.join(cfg.DATA_DIR, "temp.txt")
with open(tempfile, "w") as f:

            for sentence in tqdm(all_lines):
                
                # remove newlines
                line = sentence.strip()

                # do not save empty items such as
                if sentence != []:

                    f.writelines(sentence + '\n')

In [ ]:
import sentencepiece as spm
import time

#spm.SentencePieceTrainer.train(input=jp_sentence_files, 
#                               model_prefix='m', vocab_size=1000)

vocab_size = 32000

start = time.time()
model_prefix = "cc100_en" + "_vocab_" + str(vocab_size)
spm.SentencePieceTrainer.train(input=tempfile
                                       , model_prefix=model_prefix
                                       , vocab_size=vocab_size
                                       , character_coverage = 0.9995
                                       , num_threads=60
                                       , train_extremely_large_corpus=True
                                      ) 
print("Trained {} in {} seconds".format(model_prefix, time.time()-start))